In [10]:
import pandas as pd
import numpy as np 

In [2]:
data = pd.read_csv("./data/gemstone.csv")

In [4]:
data.drop(labels = ["id"], axis=1,inplace=True)

In [5]:
data.head()

,carat,cut,color,clarity,depth,table,x,y,z,price
0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55,13619
1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05,13387
2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50,2772
3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71,666
4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77,14453


In [3]:
x = data.drop(labels = ["price"],axis = 1)


In [22]:
y = data[["price"]]

In [23]:
y

,price
0,13619
1,13387
2,2772
3,666
4,14453
...,...
193568,1130
193569,2874
193570,3036
193571,681


In [18]:
x

,id,carat,cut,color,clarity,depth,table,x,y,z
0,0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55
1,1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05
2,2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50
3,3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71
4,4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77
...,...,...,...,...,...,...,...,...,...,...
193568,193568,0.31,Ideal,D,VVS2,61.1,56.0,4.35,4.39,2.67
193569,193569,0.70,Premium,G,VVS2,60.3,58.0,5.75,5.77,3.47
193570,193570,0.73,Very Good,F,SI1,63.1,57.0,5.72,5.75,3.62
193571,193571,0.34,Very Good,D,SI1,62.9,55.0,4.45,4.49,2.81


In [5]:
categorical_columns = x.select_dtypes(include = "object").columns

In [6]:
Numerical_columns = x.select_dtypes(exclude = "object").columns

In [8]:
cut_categories = ['Fair', 'Good', 'Very Good','Premium','Ideal']
color_categories = ['D', 'E', 'F', 'G', 'H', 'I', 'J']
clarity_categories = ['I1','SI2','SI1','VS2','VS1','VVS2','VVS1','IF']

In [9]:
from sklearn.impute import SimpleImputer ## HAndling Missing Values
from sklearn.preprocessing import StandardScaler # HAndling Feature Scaling
from sklearn.preprocessing import OrdinalEncoder # Ordinal Encoding
## pipelines
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [11]:
num_pipeline = Pipeline( 

    steps =[

        ('imputer',SimpleImputer()),
        ('scaler',StandardScaler())


    ]


)

In [13]:
cat_pipeline = Pipeline(

    steps =[
        ('imputer',SimpleImputer(strategy='most_frequent')),
        ('Ordinalencoder',OrdinalEncoder(categories=[cut_categories,color_categories,clarity_categories]))


    ]
)

In [14]:
preprocessor = ColumnTransformer(

    [
        ('num_pipeline',num_pipeline,Numerical_columns),
        ('categorical_pipeline',cat_pipeline,categorical_columns)

    ]
)

In [24]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test=train_test_split(x,y,test_size=0.30,random_state=30)

In [25]:
preprocessor.fit_transform(X_train)

array([[ 1.27491908, -0.97543926, -0.84960654, ...,  4.        ,
         5.        ,  5.        ],
       [-1.10290693,  0.2351953 ,  1.83363716, ...,  1.        ,
         1.        ,  2.        ],
       [-0.99761317,  0.49461699,  0.81585507, ...,  3.        ,
         3.        ,  4.        ],
       ...,
       [ 0.03428   ,  0.45138004,  1.55606023, ...,  1.        ,
         3.        ,  2.        ],
       [-0.87327749,  0.66756478, -1.77486298, ...,  4.        ,
         3.        ,  4.        ],
       [ 1.3030439 ,  0.25681377,  0.81585507, ...,  4.        ,
         3.        ,  2.        ]])

In [27]:
preprocessor.transform(X_test)

array([[-0.47300385, -0.56468825, -0.94213218, ...,  3.        ,
         1.        ,  3.        ],
       [-0.57300521, -0.17555571,  1.00090636, ...,  2.        ,
         4.        ,  2.        ],
       [ 1.58109167, -1.06191316,  0.2607012 , ...,  4.        ,
         4.        ,  7.        ],
       ...,
       [-0.93195878, -0.19717419, -3.34779894, ...,  3.        ,
         6.        ,  3.        ],
       [-1.198492  , -0.82410994, -0.20192702, ...,  4.        ,
         3.        ,  2.        ],
       [-0.61614902,  2.61322747, -0.75708089, ...,  3.        ,
         6.        ,  3.        ]])

In [28]:
preprocessor.get_feature_names_out()

array(['num_pipeline__id', 'num_pipeline__carat', 'num_pipeline__depth',
       'num_pipeline__table', 'num_pipeline__x', 'num_pipeline__y',
       'num_pipeline__z', 'categorical_pipeline__cut',
       'categorical_pipeline__color', 'categorical_pipeline__clarity'],
      dtype=object)

In [29]:
X_train=pd.DataFrame(preprocessor.fit_transform(X_train),columns=preprocessor.get_feature_names_out())
X_test=pd.DataFrame(preprocessor.transform(X_test),columns=preprocessor.get_feature_names_out())

In [31]:
## Model Training

from sklearn.linear_model import LinearRegression,Lasso,Ridge,ElasticNet
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error

In [32]:
import numpy as np
def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    r2_square = r2_score(true, predicted)
    return mae, rmse, r2_square

In [33]:

models={
    'LinearRegression':LinearRegression(),
    'Lasso':Lasso(),
    'Ridge':Ridge(),
    'Elasticnet':ElasticNet()
}

In [34]:
models

{'LinearRegression': LinearRegression(),
 'Lasso': Lasso(),
 'Ridge': Ridge(),
 'Elasticnet': ElasticNet()}

In [36]:
trained_model_list=[]
model_list=[]
r2_list=[]

In [38]:
for i in range(len(list(models))):
    model=list(models.values())[i]
    model.fit(X_train,y_train)

    #Make Predictions
    y_pred=model.predict(X_test)

    #this is a validation(test) score
    mae, rmse, r2_square=evaluate_model(y_test,y_pred)

    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])

    print('Model Training Performance')
    print("RMSE:",rmse)
    print("MAE:",mae)
    print("R2 score",r2_square*100)

    r2_list.append(r2_square)
    
    print('='*35)
    print('\n')

LinearRegression
Model Training Performance
RMSE: 1013.904802565844
MAE: 674.0257224619529
R2 score 93.68908132630604


Lasso
Model Training Performance
RMSE: 1013.8723151331257
MAE: 675.2036875769029
R2 score 93.68948574743231


Ridge
Model Training Performance
RMSE: 1013.9059934108715
MAE: 674.0567268148302
R2 score 93.6890665017771


Elasticnet
Model Training Performance
RMSE: 1513.91821834484
MAE: 1050.754957945767
R2 score 85.92970983426311


